In [1]:
import yfinance as yf
start_date='2020-01-01'
end_date='2023-04-05'
stocks=['ORCL','TSLA','BMY','NKE','PG','CSCO','AMD','RTX','T','ACN','XOM','AAPL','MSFT','NVDA','GOOGL','AMAT','LRCX','ZM','TDOC','CRWD',
       'MCD','JPM','JNJ','NFLX','TSM','TEAM','V','ROKU']
data = yf.download(stocks, start=start_date, end=end_date)
data.head()
import numpy as np
import pandas as pd
# Get today's date
today = '2021-03-04'
yester='2021-03-03'
# Get all stocks whose returns from yesterday's lows to today's open are lower than one standard deviation
stocks = []
for symbol in data['Adj Close'].columns:
    # Get yesterday's low and today's open prices
    low = data['Low'][[symbol]].loc[yester].values[0] if yester in data['Low'][symbol].index else np.nan
    open_price = data['Open'][[symbol]].loc[today].values[0]
    
    # Compute return from yesterday's low to today's open
    ret = np.log(open_price / low) if not np.isnan(low) and low > 0 else np.nan
    # Compute daily close-to-close returns for the last 90 days
    returns = np.log(data['Adj Close'][[symbol]] / data['Adj Close'][[symbol]].shift(1)).dropna()

    # Compute standard deviation of the returns
    std_dev = returns.std().values[0]
    # Check if return is lower than one standard deviation
    if not np.isnan(ret) and ret < -std_dev*0.1:
        stocks.append(symbol)

# Compute 20-day moving average of closing prices for each stock
closing_prices = data['Close'][stocks]
ma_20 = closing_prices.rolling(window=20).mean()

# Narrow down the list of stocks by requiring their open prices to be higher than the 20-day moving average
selected_stocks = []
for symbol in stocks:
    if data['Open'][[symbol]].loc[today].values[0] > ma_20[symbol].iloc[-1]:
        selected_stocks.append(symbol)

# Buy the 10 stocks within this list that have the lowest returns from their previous day's lows
selected_returns = []
for symbol in selected_stocks:
    # Get yesterday's low and today's open prices
    low = data['Low'][[symbol]].loc[yester].values[0] if yester in data['Low'][symbol].index else np.nan
    open_price = data['Open'][[symbol]].loc[today].values[0]
    
    # Compute return from yesterday's low to today's open
    ret = np.log(open_price / low) if not np.isnan(low) and low > 0 else np.nan
    
    # Add return to the list
    selected_returns.append(ret)

# Sort stocks by returns and select the top 10
selected_stocks = [x for _,x in sorted(zip(selected_returns,selected_stocks))][:10]

# Liquidate all positions at the market close
returns=[]
for stock in selected_stocks:
    close=data['Close'][[stock]].loc[today].values[0]
    opene=data['Open'][[stock]].loc[today].values[0]
    returns.append((close-opene)/opene)
# Not implemented, would require a different API or a time delay

# Print selected stocks
print("Selected stocks:", selected_stocks)
print('Selected returns:', returns)

[*********************100%***********************]  28 of 28 completed
Selected stocks: ['CRWD', 'TSM', 'ROKU']
Selected returns: [-0.05761994325807606, -0.05215252783524762, -0.016530021292264344]
